이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.3.1 py4j==0.10.9.5 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 12.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=a051dc3356489e2db2daffc911fcf3ba48c0630c00bea8ea3f7487a62f6cf816
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


**Spark Session**을 하나 만든다

In [85]:
from pyspark.sql import *
from pyspark.sql.functions import *

if __name__ == "__main__":
    spark = SparkSession \
        .builder \
        .appName("Spark Schema Evolution Demo") \
        .master("local[3]") \
        .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.4.1") \
        .getOrCreate()

In [86]:
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/schema1.parquet
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/schema2.parquet
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/schema3.parquet

'wget'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'wget'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'wget'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [87]:
df = spark. read \
    .format('csv') \
    .load('./data/appl_stock.csv')

In [88]:
# partitioin의 숫자 리턴
# PARTITION ID별 COUNT
print("Num Partition before : " + str(df.rdd.getNumPartitions()))
df.groupBy(spark_partition_id()).count().show()

Num Partition before : 1
+--------------------+-----+
|SPARK_PARTITION_ID()|count|
+--------------------+-----+
|                   0| 1763|
+--------------------+-----+



In [89]:
# 해싱을 통한 repartition
df2 = df.repartition(4)
print("Num Partitions after: " + str(df2.rdd.getNumPartitions()))
df2.groupBy(spark_partition_id()).count().show()

Num Partitions after: 4


+--------------------+-----+
|SPARK_PARTITION_ID()|count|
+--------------------+-----+
|                   0|  440|
|                   1|  441|
|                   2|  441|
|                   3|  441|
+--------------------+-----+



In [90]:
# coalesce라는 함수는 파티션의 수를 줄이는 함수
# 셔플링을 최소화하는 방향으로 줄인다.
df3 = df2.coalesce(2)
print("Num Partitions after: " + str(df3.rdd.getNumPartitions()))
df3.groupBy(spark_partition_id()).count().show()

Num Partitions after: 2


+--------------------+-----+
|SPARK_PARTITION_ID()|count|
+--------------------+-----+
|                   0|  881|
|                   1|  882|
+--------------------+-----+



In [91]:
# avro는 파일이 하나만 저장 
df.write \
    .format("avro") \
    .mode("overwrite") \
    .option("path", "dataOutput/avro/") \
    .save()

AnalysisException: Failed to find data source: avro. Avro is built-in but external data source module since Spark 2.4. Please deploy the application as per the deployment section of Apache Avro Data Source Guide.

In [68]:
# parquet은 파티션이 4개였고, 파일 4개
df2.write \
    .format('parquet') \
    .mode('overwrite') \
    .option('path', 'dataOutput/parquet/') \
    .save()

In [69]:
# json은 두개
df3.write \
    .format('json') \
    .mode('overwrite') \
    .option('path', 'dataOutput/json/') \
    .save()

# schema evolution

In [71]:
df1 = spark.read. \
    parquet("./data/schema1.parquet")
    
df1.printSchema()
df1.show()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)

+-------------------+------------------+------------------+------------------+------------------+
|               Date|              Open|              High|               Low|             Close|
+-------------------+------------------+------------------+------------------+------------------+
|2010-01-04 17:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|
|2010-01-05 17:00:00|        214.599998|        215.589994|        213.249994|        214.379993|
|2010-01-06 17:00:00|        214.379993|            215.23|        210.750004|        210.969995|
|2010-01-07 17:00:00|            211.75|        212.000006|        209.050005|            210.58|
|2010-01-08 17:00:00|        210.299994|        212.000006|209.06000500000002|211.98000499999998|
|2010-01-11 17:00:00|212.79

In [72]:
df2 = spark.read. \
    parquet("./data/schema2.parquet")
    
df2.printSchema()
df2.show()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)

+-------------------+------------------+------------------+------------------+------------------+---------+
|               Date|              Open|              High|               Low|             Close|   Volume|
+-------------------+------------------+------------------+------------------+------------------+---------+
|2010-01-04 17:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|
|2010-01-05 17:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|
|2010-01-06 17:00:00|        214.379993|            215.23|        210.750004|        210.969995|138040000|
|2010-01-07 17:00:00|            211.75|        212.000006|        209.050005|            210.58|119282800|
|2010-01-08 17:0

In [74]:
df3 = spark.read. \
    parquet("./data/schema3.parquet")
    
df3.printSchema()
df3.show()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 17:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 17:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 17:00:00|        214.379993|            215.23|        210.750004|        210.969995

In [76]:
# schema 파일을 동시에 합쳐서 데이터를 합칠 수 있다.
# mergeSchema를 트루로 지정
df = spark.read. \
    option("mergeSchema", True). \
    parquet("./data/*.parquet")
    
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [77]:
df.show(10)

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 17:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 17:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 17:00:00|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 17:00:00|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 17:00:00|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|    